# Get MF-DFA on starling amplitude envelope
#### Date: 8/31/2022, JX
> Use MF-DFA on /mnt/cube/j8xing/starling_rhythm_mfdfa/data/starling_zenodo_2019_amp_envs.pickle

In [1]:
from starling_rhythm.utils.paths import PROCESSED_DIR

## Import dataframe

In [2]:
import pandas as pd

In [3]:
SAVE_PATH = PROCESSED_DIR / 'starling_b1555_amp_envs200_spl.pickle'
ae_df = pd.read_pickle(SAVE_PATH)

In [4]:
ae_df

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing,is_call,amp_env,spl_env
0,508500,566065,24408000,27171120,"[28.88509981238539, 65.14898407709249, 103.180...",9.609107,143.211264,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,57565,...,472,121.959746,"[-5, -5, -5, -5, -3, -5, -5, -4, -6, -4, -2, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[2.4892911824281327e-05, 0.002640440361574292,...","[0.012887999192663964, 1.6298831714527595, 3.0..."
1,641180,694125,30776640,33318000,"[48.20694858466058, 83.76981436987464, 108.852...",5.388905,156.745806,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,52945,...,555,95.396396,"[3, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, -1, -1, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[5.805372893519234e-06, 0.0016227918677031994,...","[0.00406788556508693, 1.1053593978847633, 2.39..."
2,868940,921030,41709120,44209440,"[2.3001409701376363, 15.027985352733767, 30.17...",8.135380,156.958417,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,52090,...,400,130.225000,"[-11, -11, -11, -11, -12, -11, -11, -11, -10, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[9.765496361069381e-05, 0.007925855927169323, ...","[0.025815673495165517, 3.3906952118208697, 5.3..."
3,1043035,1093720,50065680,52498560,"[2.9249842953419494, 3.1541257656248125, 14.16...",8.135380,144.054406,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,50685,...,276,183.641304,"[-2, -2, -1, -2, -2, -2, -3, -3, -3, -3, -3, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[5.769828931079246e-05, 0.015915358439087868, ...","[0.020930336806865055, 5.403901915623869, 11.0..."
4,1423905,1474480,68347440,70775040,"[3.937345912388517, 3.469701876813218, 2.86553...",4.015413,140.816750,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,50575,...,641,78.900156,"[6, 5, 5, 5, 3, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[5.337237780622672e-06, 0.0012986172223463655,...","[0.00395726763677555, 0.9592529687682605, 1.81..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,1591530,1602065,76393440,76899120,"[4.643018462812651, 2.923919620954338, 13.5363...",2.043606,114.653102,False,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,10535,...,51,206.568627,"[-6, -8, -9, -6, -4, -5, -8, -10, -8, -6, -6, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[3.023368481080979e-05, 0.0012741615064442158,...","[0.017286074782681404, 1.0340200170418643, 2.0..."
123,1605580,1610770,77067840,77316960,"[33.25759392172391, 29.86369699500519, 6.09811...",2.043606,238.422834,False,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,5190,...,39,133.076923,"[12, 13, 12, 12, 12, 12, 14, 14, 14, 13, 12, 1...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[7.692791405133903e-05, 0.018301259726285934, ...","[0.019650267776198868, 4.695249442450509, 9.37..."
136,411070,415280,19731360,19933440,"[5.175037749402548, 70.80952142186685, 57.2026...",2.616865,159.078826,False,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,4210,...,29,145.172414,"[12, 9, 5, 4, 4, 6, 10, 10, 11, 13, 16, 16, 13...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[1.1144771633553319e-05, 0.002191337989643216,...","[0.007439472871148186, 1.5052592241305685, 4.3..."
145,1620240,1623675,77771520,77936400,"[26.356576704532092, 27.208591690945624, 8.918...",2.125249,46.728800,False,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,3435,...,12,286.250000,"[-4, -5, -6, -6, -5, -2, -3, -5, -6, -9, -9, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[7.4280251283198595e-06, 0.02198933996260166, ...","[0.008292225865660021, 9.1

## Pull out MF-DFA

> 1. Filter time-series to have compact support
2. Determine time windows

In [5]:
import numpy as np
from starling_rhythm.mfdfa import hurst_expo
from joblib import Parallel, delayed
from tqdm.autonotebook import tqdm

/home/AD/j8xing/anaconda3/envs/starling_rhythm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
n_jobs = 36 ## set up joblib parameters
verbose = 0

In [7]:
with Parallel(n_jobs = n_jobs, verbose = verbose) as parallel:
    h_expos_ae = parallel(
        delayed(hurst_expo)(
            amp_env
        ) 
        for amp_env in tqdm(ae_df.amp_env.values, desc = "Retrieving Hurst Exponents for all Amplitude Envelopes.")
    )
    
with Parallel(n_jobs = n_jobs, verbose = verbose) as parallel:
    h_expos_spl = parallel(
        delayed(hurst_expo)(
            amp_env
        ) 
        for amp_env in tqdm(ae_df.spl_env.values, desc = "Retrieving Hurst Exponents for all SPL Envelopes.")
    )

Retrieving Hurst Exponents for all Amplitude Envelopes.: 100%|██████████| 1041/1041 [00:39<00:00, 26.34it/s]
Retrieving Hurst Exponents for all SPL Envelopes.: 100%|██████████| 1041/1041 [00:16<00:00, 61.88it/s]


In [8]:
## Get MF_range
MF_range_ae = []
for h_expo in h_expos_ae:
    MF_range_ae.append(max(h_expo) - min(h_expo))
    
MF_range_spl = []
for h_expo in h_expos_spl:
    MF_range_spl.append(max(h_expo) - min(h_expo))

In [9]:
ae_df['h_expo_ae'] = h_expos_ae
ae_df['MF_range_ae'] = MF_range_ae
ae_df['h_expo_spl'] = h_expos_spl
ae_df['MF_range_spl'] = MF_range_spl

In [10]:
ae_df

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,valid,spectrogram,confusing,is_call,amp_env,spl_env,h_expo_ae,MF_range_ae,h_expo_spl,MF_range_spl
0,508500,566065,24408000,27171120,"[28.88509981238539, 65.14898407709249, 103.180...",9.609107,143.211264,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,57565,...,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[2.4892911824281327e-05, 0.002640440361574292,...","[0.012887999192663964, 1.6298831714527595, 3.0...","[1.210268876562112, 1.0847868581968103, 0.9997...",0.417653,"[1.0488658170321583, 1.0192658050285757, 0.997...",0.137107
1,641180,694125,30776640,33318000,"[48.20694858466058, 83.76981436987464, 108.852...",5.388905,156.745806,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,52945,...,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[5.805372893519234e-06, 0.0016227918677031994,...","[0.00406788556508693, 1.1053593978847633, 2.39...","[1.206747469037927, 1.0668692741274823, 0.9747...",0.461689,"[1.0845706146074268, 1.0475678318350459, 1.022...",0.152306
2,868940,921030,41709120,44209440,"[2.3001409701376363, 15.027985352733767, 30.17...",8.135380,156.958417,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,52090,...,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[9.765496361069381e-05, 0.007925855927169323, ...","[0.025815673495165517, 3.3906952118208697, 5.3...","[1.2857270505789482, 1.1416211034999122, 1.035...",0.537401,"[1.054714273517234, 1.0216348378749254, 0.9976...",0.155954
3,1043035,1093720,50065680,52498560,"[2.9249842953419494, 3.1541257656248125, 14.16...",8.135380,144.054406,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,50685,...,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[5.769828931079246e-05, 0.015915358439087868, ...","[0.020930336806865055, 5.403901915623869, 11.0...","[1.317296971969834, 1.1366829235572742, 1.0264...",0.522708,"[1.1249544237114795, 1.0707444454679387, 1.032...",0.227368
4,1423905,1474480,68347440,70775040,"[3.937345912388517, 3.469701876813218, 2.86553...",4.015413,140.816750,True,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,50575,...,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[5.337237780622672e-06, 0.0012986172223463655,...","[0.00395726763677555, 0.9592529687682605, 1.81...","[1.1679092266897195, 1.0605409985158545, 0.984...",0.388723,"[0.990100912082668, 0.9594052503013833, 0.9354...",0.152659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,1591530,1602065,76393440,76899120,"[4.643018462812651, 2.923919620954338, 13.5363...",2.043606,114.653102,False,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,10535,...,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[3.023368481080979e-05, 0.0012741615064442158,...","[0.017286074782681404, 1.0340200170418643, 2.0...","[1.2917844908328622, 1.1501793367891089, 1.021...",0.659497,"[1.1744464301571218, 1.1276826778296476, 1.096...",0.184049
123,1605580,1610770,77067840,77316960,"[33.25759392172391, 29.86369699500519, 6.09811...",2.043606,238.422834,False,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,5190,...,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[7.692791405133903e-05, 0.018301259726285934, ...","[0.019650267776198868, 4.695249442450509, 9.37...","[1.7012421954121975, 1.3053615144730153, 1.059...",1.050222,"[1.3893312733498826, 1.3055277312161337, 1.250...",0.308968
136,411070,415280,19731360,19933440,"[5.175037749402548, 70.80952142186685, 57.2026...",2.616865,159.078826,False,/mnt/sphere/speech_bci/raw_data/s_b1555_22/202...,4210,...,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False,False,"[1.1144771633553319e-05, 0.002191337989643216,...","[0.007439472871148186, 1.5052592241305685, 4.3...","[1.5564953817880078, 1.2752443607114707, 1.055...",0.935401,"[1.3317134342289572, 1.2596379996286222, 1.2

## Pickle dataframe

In [11]:
from starling_rhythm.utils.paths import PROCESSED_DIR, ensure_dir

In [12]:
SAVE_PATH = PROCESSED_DIR / 'starling_b1555_amp_envs200sr_MF_spl.pickle'
ensure_dir(SAVE_PATH)

In [13]:
ae_df.to_pickle(SAVE_PATH)